### Spam SMS data analysis

Dataset info: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset

In [26]:
import pandas as pd
import nltk
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from spacy.lang.he import Hebrew
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import requests
#TODO: install pytorch?
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
# load the data
spam_data = pd.read_csv('spam.csv', encoding='latin-1')
spam_data.head(10)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
7,ham,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN
8,spam,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN


In [4]:
# drop unnecessary columns
spam_data = spam_data[['v1', 'v2']]

In [5]:
# Number of sms messages
num_messages = spam_data.shape[0]
print("We have a total of {} messages".format(num_messages))
# map ham to 0 and spam to 1
spam_data['v1'] = spam_data['v1'].map({'ham': 0, 'spam': 1})
spam_data.head(10)

We have a total of 5572 messages


,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
5,1,FreeMsg Hey there darling it's been 3 week's n...
6,0,Even my brother is not like to speak with me. ...
7,0,As per your request 'Melle Melle (Oru Minnamin...
8,1,WINNER!! As a valued network customer you have...
9,1,Had your mobile 11 months or more? U R entitle...


In [6]:
# Compute number of spam messages
num_spam = spam_data['v1'].sum()
print("We have a total of {} spam messages".format(num_spam))
messages = spam_data['v2'].values
num_of_words = sum([len(message.split(" ")) for message in messages])
print("We have a total of {} words".format(num_of_words))
avg_num_words = num_of_words / num_messages
print("On average, we have {} words per message".format(avg_num_words))

We have a total of 747 spam messages
We have a total of 86961 words
On average, we have 15.60678391959799 words per message


In [7]:
# all words counter
words_counter = {}
for message in messages:
    for word in message.split(" "):
        if word not in words_counter:
            words_counter[word] = 1
        else:
            words_counter[word] += 1
# print 5 most common words
print("5 most common words:")
print(sorted(words_counter.items(), key=lambda x: x[1], reverse=True)[:5])

appear_once_words = [word for word, count in words_counter.items() if count == 1]
print("Number of words that appear once: {}".format(len(appear_once_words)))

5 most common words:
[('to', 2134), ('you', 1622), ('I', 1466), ('a', 1327), ('the', 1197)]
Number of words that appear once: 9270


### Tokenization: NLTK VS Spacy

In [ ]:
def NLTK_tokenize(messages):
    all_messages = " ".join(messages)
    # perform tokenization using nltk
    res = nltk.word_tokenize(all_messages)
    return res 
nltk_tokenized = NLTK_tokenize(messages)
print(nltk_tokenized)

['Go', 'until', 'jurong', 'point', ',', 'crazy', '..', 'Available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'Cine', 'there', 'got', 'amore', 'wat', '...', 'Ok', 'lar', '...', 'Joking', 'wif', 'u', 'oni', '...', 'Free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'FA', 'Cup', 'final', 'tkts', '21st', 'May', '2005', '.', 'Text', 'FA', 'to', '87121', 'to', 'receive', 'entry', 'question', '(', 'std', 'txt', 'rate', ')', 'T', '&', 'C', "'s", 'apply', '08452810075over18', "'s", 'U', 'dun', 'say', 'so', 'early', 'hor', '...', 'U', 'c', 'already', 'then', 'say', '...', 'Nah', 'I', 'do', "n't", 'think', 'he', 'goes', 'to', 'usf', ',', 'he', 'lives', 'around', 'here', 'though', 'FreeMsg', 'Hey', 'there', 'darling', 'it', "'s", 'been', '3', 'week', "'s", 'now', 'and', 'no', 'word', 'back', '!', 'I', "'d", 'like', 'some', 'fun', 'you', 'up', 'for', 'it', 'still', '?', 'Tb', 'ok', '!', 'XxX', 'std', 'chgs', 'to', 'send', ',', 'å£1.50', 'to', 'rcv', 'Even'

In [53]:
def spacy_tokenize(messages, lang='en'):
    all_messages = " ".join(messages)
    if lang == 'en':
        nlp = English()
    elif lang == 'he':
        nlp = Hebrew()
    tokenizer = Tokenizer(nlp.vocab)
    return tokenizer(all_messages)
spacy_tokenized = spacy_tokenize(messages)
print(list(spacy_tokenized)[0:120])

[Go, until, jurong, point,, crazy.., Available, only, in, bugis, n, great, world, la, e, buffet..., Cine, there, got, amore, wat..., Ok, lar..., Joking, wif, u, oni..., Free, entry, in, 2, a, wkly, comp, to, win, FA, Cup, final, tkts, 21st, May, 2005., Text, FA, to, 87121, to, receive, entry, question(std, txt, rate)T&C's, apply, 08452810075over18's, U, dun, say, so, early, hor..., U, c, already, then, say..., Nah, I, don't, think, he, goes, to, usf,, he, lives, around, here, though, FreeMsg, Hey, there, darling, it's, been, 3, week's, now, and, no, word, back!, I'd, like, some, fun, you, up, for, it, still?, Tb, ok!, XxX, std, chgs, to, send,, å£1.50, to, rcv, Even, my, brother, is, not, like, to, speak, with, me.]


### Question 5
The main differences we have spotted are:
1. nltk tokenizer takes punctuation as separated tokens, while spacy's tokenizer concat them to the nearest word
2. spacy's tokenizer chose to remain words with ' like "don't" as a whole token, while nltk tokenizer separated it to 'do' and "n't"

### lemmatization: NLTK VS Spacy

In [10]:
# perform lemmatization using nltk
def NLTK_lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]

nltk_lemmatized = NLTK_lemmatize(nltk_tokenized)
print(nltk_lemmatized[0:120])

['Go', 'until', 'jurong', 'point', ',', 'crazy', '..', 'Available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'Cine', 'there', 'got', 'amore', 'wat', '...', 'Ok', 'lar', '...', 'Joking', 'wif', 'u', 'oni', '...', 'Free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'FA', 'Cup', 'final', 'tkts', '21st', 'May', '2005', '.', 'Text', 'FA', 'to', '87121', 'to', 'receive', 'entry', 'question', '(', 'std', 'txt', 'rate', ')', 'T', '&', 'C', "'s", 'apply', '08452810075over18', "'s", 'U', 'dun', 'say', 'so', 'early', 'hor', '...', 'U', 'c', 'already', 'then', 'say', '...', 'Nah', 'I', 'do', "n't", 'think', 'he', 'go', 'to', 'usf', ',', 'he', 'life', 'around', 'here', 'though', 'FreeMsg', 'Hey', 'there', 'darling', 'it', "'s", 'been', '3', 'week', "'s", 'now', 'and', 'no', 'word', 'back', '!', 'I', "'d", 'like', 'some', 'fun', 'you', 'up']


In [54]:
# perform lemmatization using spacy
# TODO replace messages with tokens
def spacy_lemmatize(messages):
    spacy_lemmatizer = spacy.load('en_core_web_sm')
    lemmatized_words = [
        token.lemma_
        for message in messages
        for token in spacy_lemmatizer(message)
    ]
    return lemmatized_words

In [ ]:
spacy_lemmatized = spacy_lemmatize(messages)
print(spacy_lemmatized[0:120])

In [12]:
# compare the two lemmatized lists
print("NLTK lemmatized words:")
print(nltk_lemmatized[:120])
print("Length of NLTK lemmatized words:")
print(len(nltk_lemmatized))
print("Spacy lemmatized words:")
print(spacy_lemmatized[:120])
print("Length of Spacy lemmatized words:")
print(len(spacy_lemmatized))

NLTK lemmatized words:
['Go', 'until', 'jurong', 'point', ',', 'crazy', '..', 'Available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'Cine', 'there', 'got', 'amore', 'wat', '...', 'Ok', 'lar', '...', 'Joking', 'wif', 'u', 'oni', '...', 'Free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'FA', 'Cup', 'final', 'tkts', '21st', 'May', '2005', '.', 'Text', 'FA', 'to', '87121', 'to', 'receive', 'entry', 'question', '(', 'std', 'txt', 'rate', ')', 'T', '&', 'C', "'s", 'apply', '08452810075over18', "'s", 'U', 'dun', 'say', 'so', 'early', 'hor', '...', 'U', 'c', 'already', 'then', 'say', '...', 'Nah', 'I', 'do', "n't", 'think', 'he', 'go', 'to', 'usf', ',', 'he', 'life', 'around', 'here', 'though', 'FreeMsg', 'Hey', 'there', 'darling', 'it', "'s", 'been', '3', 'week', "'s", 'now', 'and', 'no', 'word', 'back', '!', 'I', "'d", 'like', 'some', 'fun', 'you', 'up']
Length of NLTK lemmatized words:
104164
Spacy lemmatized words:
['go', 'until', 'jurong', 'poi

### Question 6
The main differnces are: 
1. Running time: Spacy is much slower due to running ML models in order to return a more accurate results. For instance, the POS tagging, which predicts the grammatical role of each word (noun, verb, etc..)
2. nltk lemmatizer returned words like "it's" while spacy splitted it to "it", "be" ('s -> is -> be).
3. nltk's lemmatizer didn't lemmatize the work 'joking' (stayed the same) while spacy's lemmatizer change it to 'joke'. 
4. nltk's returned the 'me' while spacy returned 'i'.

### Stemming: NLTK VS Spacy

In [13]:
def NLTK_stem(tokens):
    stemmer = nltk.LancasterStemmer()
    return [stemmer.stem(token) for token in tokens]

# nltk stemming
stemmed_words_nltk = NLTK_stem(nltk_lemmatized)
#spacy stemming
stemmed_words_spacy = NLTK_stem(spacy_lemmatized)

In [14]:
print("NLTK stemming")
print(stemmed_words_nltk[:120])
print("Length of NLTK stemmed words: {}".format(len(stemmed_words_nltk)))
print("Spacy stemming")
print(stemmed_words_spacy[:120])
print("Length of Spacy stemmed words: {}".format(len(stemmed_words_spacy)))

NLTK stemming
['go', 'until', 'jurong', 'point', ',', 'crazy', '..', 'avail', 'on', 'in', 'bug', 'n', 'gre', 'world', 'la', 'e', 'buffet', '...', 'cin', 'ther', 'got', 'am', 'wat', '...', 'ok', 'lar', '...', 'jok', 'wif', 'u', 'on', '...', 'fre', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'fa', 'cup', 'fin', 'tkts', '21st', 'may', '2005', '.', 'text', 'fa', 'to', '87121', 'to', 'receiv', 'entry', 'quest', '(', 'std', 'txt', 'rat', ')', 't', '&', 'c', "'s", 'apply', '08452810075over18', "'s", 'u', 'dun', 'say', 'so', 'ear', 'hor', '...', 'u', 'c', 'already', 'then', 'say', '...', 'nah', 'i', 'do', "n't", 'think', 'he', 'go', 'to', 'usf', ',', 'he', 'lif', 'around', 'her', 'though', 'freemsg', 'hey', 'ther', 'darl', 'it', "'s", 'been', '3', 'week', "'s", 'now', 'and', 'no', 'word', 'back', '!', 'i', "'d", 'lik', 'som', 'fun', 'you', 'up']
Length of NLTK stemmed words: 104164
Spacy stemming
['go', 'until', 'jurong', 'point', ',', 'crazy', '..', 'avail', 'on', 'in', 'bug', 'n', 

### Question 7
There is no significant differences between the two results. 
We have noticed that the nltk's LancasterStemmer has lower cased all word's first letter. 

In [15]:
# Loop over all the mesages if the message isnt spam - than add all the tokens of llematized and stemming that didnt appear in the arrays(stemming tokens and llematized tokens)
# if the message is spam - than check if all tokens are in the stemming and lemmatized arrays

# first - we will save a dict of all the tokens that are in the lemmatized and stemming arrays
nltk_stemmer = nltk.LancasterStemmer()
nltk_lemmatizer = WordNetLemmatizer()
def count_lemmatized_and_stemmed(messages):
    lemmatized_dict = {}
    stemmed_dict = {}
    for message in messages:
        stem_message = nltk_stemmer.stem(message)
        lemmatized_message = nltk_lemmatizer.lemmatize(message)
        for token in stem_message.split(" "):
            if token not in stemmed_dict:
                stemmed_dict[token] = 1
            else:
                stemmed_dict[token] += 1
        for token in lemmatized_message.split(" "):
            if token not in lemmatized_dict:
                lemmatized_dict[token] = 1
            else:
                lemmatized_dict[token] += 1
    print(lemmatized_dict)
    print(stemmed_dict)
    return lemmatized_dict, stemmed_dict

## Question 8

In [16]:
# now we will loop over all the messages and check if the message is spam
# if the message is spam - than check if all its tokens are in the stemming more than 1 time and lemmatized only 1 time
spam_messages = spam_data[spam_data['v1'] == 1]['v2'].values
def reduce_stem_maintain_lemmatized(messages, is_sms=False):
    lemmatized_dict, stemmed_dict = count_lemmatized_and_stemmed(messages)
    if is_sms: 
        messages = spam_messages
    for message in messages:
        can_remove = True
        stem_message = nltk_stemmer.stem(message)
        lemmatized_message = nltk_lemmatizer.lemmatize(message)
        # check if all the tokens are in the stemming and lemmatized arrays
        for token in stem_message.split(" "):
            if stemmed_dict[token] != 1:
                can_remove = False
                break
        for token in lemmatized_message.split(" "):
            if lemmatized_dict[token] == 1:
                can_remove = False
                break
        if can_remove:
            print("can remove this message")
            print(message)

reduce_stem_maintain_lemmatized(messages, True)

{'Go': 14, 'until': 26, 'jurong': 1, 'point,': 1, 'crazy..': 1, 'Available': 3, 'only': 147, 'in': 798, 'bugis': 4, 'n': 137, 'great': 75, 'world': 17, 'la': 2, 'e': 71, 'buffet...': 1, 'Cine': 1, 'there': 118, 'got': 204, 'amore': 1, 'wat...': 12, 'Ok': 97, 'lar...': 18, 'Joking': 1, 'wif': 26, 'u': 692, 'oni...': 2, 'Free': 39, 'entry': 25, '2': 453, 'a': 1327, 'wkly': 10, 'comp': 10, 'to': 2134, 'win': 38, 'FA': 4, 'Cup': 3, 'final': 13, 'tkts': 4, '21st': 3, 'May': 16, '2005.': 3, 'Text': 42, '87121': 4, 'receive': 34, 'question(std': 2, 'txt': 75, "rate)T&C's": 2, 'apply': 15, "08452810075over18's": 2, 'U': 299, 'dun': 46, 'say': 72, 'so': 294, 'early': 24, 'hor...': 2, 'c': 44, 'already': 45, 'then': 146, 'say...': 3, 'Nah': 7, 'I': 1466, "don't": 110, 'think': 113, 'he': 127, 'goes': 26, 'usf,': 2, 'lives': 2, 'around': 53, 'here': 75, 'though': 15, 'FreeMsg': 5, 'Hey': 76, 'darling': 4, "it's": 53, 'been': 120, '3': 51, "week's": 4, 'now': 190, 'and': 857, 'no': 124, 'word': 22

## Question 9

In [17]:
def reduce_lemmatized_maintain_stemmed(messages, is_sms=False):
    lemmatized_dict, stemmed_dict = count_lemmatized_and_stemmed(messages)
    if is_sms: 
        messages = spam_messages
    for message in messages:
        can_remove = True
        stem_message = nltk_stemmer.stem(message)
        lemmatized_message = nltk_lemmatizer.lemmatize(message)
        # check if all the tokens are in the stemming and lemmatized arrays
        for token in stem_message.split(" "):
            if stemmed_dict[token] == 1:
                can_remove = False
                break
        for token in lemmatized_message.split(" "):
            if lemmatized_dict[token] != 1:
                can_remove = False
                break
        if can_remove:
            print("can remove this message")
            print(message)
reduce_lemmatized_maintain_stemmed(messages, True)

{'Go': 14, 'until': 26, 'jurong': 1, 'point,': 1, 'crazy..': 1, 'Available': 3, 'only': 147, 'in': 798, 'bugis': 4, 'n': 137, 'great': 75, 'world': 17, 'la': 2, 'e': 71, 'buffet...': 1, 'Cine': 1, 'there': 118, 'got': 204, 'amore': 1, 'wat...': 12, 'Ok': 97, 'lar...': 18, 'Joking': 1, 'wif': 26, 'u': 692, 'oni...': 2, 'Free': 39, 'entry': 25, '2': 453, 'a': 1327, 'wkly': 10, 'comp': 10, 'to': 2134, 'win': 38, 'FA': 4, 'Cup': 3, 'final': 13, 'tkts': 4, '21st': 3, 'May': 16, '2005.': 3, 'Text': 42, '87121': 4, 'receive': 34, 'question(std': 2, 'txt': 75, "rate)T&C's": 2, 'apply': 15, "08452810075over18's": 2, 'U': 299, 'dun': 46, 'say': 72, 'so': 294, 'early': 24, 'hor...': 2, 'c': 44, 'already': 45, 'then': 146, 'say...': 3, 'Nah': 7, 'I': 1466, "don't": 110, 'think': 113, 'he': 127, 'goes': 26, 'usf,': 2, 'lives': 2, 'around': 53, 'here': 75, 'though': 15, 'FreeMsg': 5, 'Hey': 76, 'darling': 4, "it's": 53, 'been': 120, '3': 51, "week's": 4, 'now': 190, 'and': 857, 'no': 124, 'word': 22

## Question 10

In [18]:
def compare_nltk_results(messages):
    tokenized = NLTK_tokenize(messages)
    lemmatized = NLTK_lemmatize(tokenized)
    stemmed = NLTK_stem(lemmatized)
    # compare the three lists
    # unique words in tokenized
    unique_tokenized = set(tokenized)
    unique_lemmatized = set(lemmatized)
    unique_stemmed = set(stemmed)
    print("Tokens: {}".format(tokenized[:120]))
    print("Lemmatized words: {}".format(lemmatized[:120]))
    print("Stemmed words: {}".format(stemmed[:120]))
    print("Total number of words in tokenized: {}".format(len(tokenized)))
    print("Total number of words in lemmatized: {}".format(len(lemmatized)))
    print("Total number of words in stemmed: {}".format(len(stemmed)))
    print("Unique words in tokenized: {}".format(len(unique_tokenized)))
    print("Unique words in lemmatized: {}".format(len(unique_lemmatized)))
    print("Unique words in stemmed: {}".format(len(unique_stemmed)))

compare_nltk_results(messages)

Tokens: ['Go', 'until', 'jurong', 'point', ',', 'crazy', '..', 'Available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'Cine', 'there', 'got', 'amore', 'wat', '...', 'Ok', 'lar', '...', 'Joking', 'wif', 'u', 'oni', '...', 'Free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'FA', 'Cup', 'final', 'tkts', '21st', 'May', '2005', '.', 'Text', 'FA', 'to', '87121', 'to', 'receive', 'entry', 'question', '(', 'std', 'txt', 'rate', ')', 'T', '&', 'C', "'s", 'apply', '08452810075over18', "'s", 'U', 'dun', 'say', 'so', 'early', 'hor', '...', 'U', 'c', 'already', 'then', 'say', '...', 'Nah', 'I', 'do', "n't", 'think', 'he', 'goes', 'to', 'usf', ',', 'he', 'lives', 'around', 'here', 'though', 'FreeMsg', 'Hey', 'there', 'darling', 'it', "'s", 'been', '3', 'week', "'s", 'now', 'and', 'no', 'word', 'back', '!', 'I', "'d", 'like', 'some', 'fun', 'you', 'up']
Lemmatized words: ['Go', 'until', 'jurong', 'point', ',', 'crazy', '..', 'Available', 'only', 'in', 'bugis

## Question 11

In [19]:
def compare_spacy_results(messages):
    tokenized = spacy_tokenize(messages)
    lemmatized = spacy_lemmatize(messages)
    # compare the two lists
    # unique words in tokenized
    unique_tokenized = set(tokenized)
    unique_lemmatized = set(lemmatized)
    print("Tokens: {}".format(tokenized[:120]))
    print("Lemmatized words: {}".format(lemmatized[:120]))
    print("Total number of words in tokenized: {}".format(len(tokenized)))
    print("Total number of words in lemmatized: {}".format(len(lemmatized)))
    print("Unique words in tokenized: {}".format(len(unique_tokenized)))
    print("Unique words in lemmatized: {}".format(len(unique_lemmatized)))

compare_spacy_results(messages)

Tokens: Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat... Ok lar... Joking wif u oni... Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's U dun say so early hor... U c already then say... Nah I don't think he goes to usf, he lives around here though FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv Even my brother is not like to speak with me.
Lemmatized words: ['go', 'until', 'jurong', 'point', ',', 'crazy', '..', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'Cine', 'there', 'get', 'amore', 'wat', '...', 'ok', 'lar', '...', 'joke', 'wif', 'u', 'oni', '...', 'free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'FA', 'Cup', 'final', 'tkts', '21st', 'May', '2005', '.', 'text'

In [22]:
# scrapping from google wiki page

url = "https://en.wikipedia.org/wiki/Meta_Platforms"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
text = [p.text.replace('\n', '') for p in soup.find_all("p")]
text = [t.replace('\n', '') for t in text]
text = [t for t in text if len(t) > 0]
print(text) 

['Meta Platforms, Inc.,[9] doing business as Meta,[10] and formerly named Facebook, Inc., and The Facebook, Inc.,[11][12] is an American multinational technology conglomerate based in Menlo Park, California. The company owns and operates Facebook, Instagram, Threads, and WhatsApp, among other products and services.[13] Advertising accounts for 97.8 percent of its revenue.[a][14] Originally known as the parent company of the Facebook service, as Facebook, Inc., it was rebranded to its current name in 2021 to "reflect its focus on building the metaverse",[15] an integrated environment linking the company\'s products and services.[16][17][18]', 'Meta ranks among the largest American information technology companies, alongside the other Big Five corporations: Alphabet (Google), Amazon, Apple, and Microsoft. In 2023 the company was ranked #31 on the Forbes Global 2000 ranking.[19] In 2022, Meta was the company with the third-highest expenditure on research and development worldwide, with R&

In [55]:
def perform_5_to_11(text, language='en'):
    # perform tokenization using nltk
    nltk_tokenized = NLTK_tokenize(text)
    print("nltk tokenized:" + str(nltk_tokenized))
    spacy_tokenized = spacy_tokenize(text, language)
    print("spacy tokenized:" + str(spacy_tokenized))
    # perform lemmatization using nltk
    nltk_lemmatized = NLTK_lemmatize(nltk_tokenized)
    print("nltk lemmatized:" + str(nltk_lemmatized))
    # perform lemmatization using spacy
    spacy_lemmatized = spacy_lemmatize(text)
    print("spacy lemmatized:" + str(spacy_lemmatized))
    # perform stemming using nltk
    nltk_stemmed = NLTK_stem(nltk_lemmatized)
    print("nltk stemmed:" + str(nltk_stemmed))
    # perform stemming using spacy
    spacy_stemmed = NLTK_stem(spacy_lemmatized)
    print("spacy stemmed:" + str(spacy_stemmed))
    reduce_stem_maintain_lemmatized(text)
    reduce_lemmatized_maintain_stemmed(text)
    compare_nltk_results(text)
    compare_spacy_results(text)


In [ ]:
# perform the steps on the wiki page
perform_5_to_11(text)

In [24]:
# load all the text from the whatsapp chat

with open('whatsapp_chat.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines if line.strip()]
    lines = [line.replace('\n', '') for line in lines]
    lines = [line.replace('\r', '') for line in lines]
    lines = [line.replace('\t', '') for line in lines]
    whatsapp_text = [line for line in lines if len(line) > 0]
print(whatsapp_text)

['13.12.2024, 13:43 - שלום לכולם מדבר בן אילתי', 'ודן ביקש ממני שנתכנס איזה יום אחד ראשי הצוותים לדבר על המשימות של המועדון', '13.12.2024, 16:40 -  ערב טוב, המטרה היא לקיים דיון שבועי בו נעשה חיתוך מצב ברמה הטכנית בניגוד לדיון של יום שלישי. כל ראש צוות יציג את ההתקדמות האתגרים ברמה הטכנית', 'יום לפגישה', 'שעות', '15.12.2024, 11:28 -  הכוונה לשבת כחול הנראה שעה ולא שעתיים', '16.12.2024, 16:25 -  איפה אני מוצא את התקנון הכי עדכני של התחרות', '16.12.2024, 16:26 -  ולפי מה עדיף ללכת, זה של שנה הבאה באוסטרליה או זה של השנה בבלגיה', '16.12.2024, 16:45 -  <המדיה לא נכללה>', '16.12.2024, 18:03 -  אוסטרליה 25.', 'את התקנון שאנחנו נתחרה איתו יפרסמו בינואר 26 והוא יהיה מבוסס על האוסטרלי', '16.12.2024, 18:12 -  כי הכלים שלהם קצת חסומים כאשר ניסיתי להריץ את השלדה שלנו', '16.12.2024, 18:14 -  מעולה', '17.12.2024, 15:48 -  האם, כל מי שבקבוצה מגיע היום ?', '17.12.2024, 16:15 -  הפגישה היום תתקיים במתכונת של ראשי צוותים.', '18.12.2024, 16:48 - היי בן ודן', 'מתי מחר אתם פנויים לפגישת צלילות ?', 'אני ב10

## Question 17

### nltk and spacy don't support lemmatiziation nor stemming in Hebrew

In [56]:
# perform steps 5-11 on the whatsapp chat
perform_5_to_11(whatsapp_text, 'he')

nltk tokenized:['13.12.2024', ',', '13:43', '-', 'שלום', 'לכולם', 'מדבר', 'בן', 'אילתי', 'ודן', 'ביקש', 'ממני', 'שנתכנס', 'איזה', 'יום', 'אחד', 'ראשי', 'הצוותים', 'לדבר', 'על', 'המשימות', 'של', 'המועדון', '13.12.2024', ',', '16:40', '-', 'ערב', 'טוב', ',', 'המטרה', 'היא', 'לקיים', 'דיון', 'שבועי', 'בו', 'נעשה', 'חיתוך', 'מצב', 'ברמה', 'הטכנית', 'בניגוד', 'לדיון', 'של', 'יום', 'שלישי', '.', 'כל', 'ראש', 'צוות', 'יציג', 'את', 'ההתקדמות', 'האתגרים', 'ברמה', 'הטכנית', 'יום', 'לפגישה', 'שעות', '15.12.2024', ',', '11:28', '-', 'הכוונה', 'לשבת', 'כחול', 'הנראה', 'שעה', 'ולא', 'שעתיים', '16.12.2024', ',', '16:25', '-', 'איפה', 'אני', 'מוצא', 'את', 'התקנון', 'הכי', 'עדכני', 'של', 'התחרות', '16.12.2024', ',', '16:26', '-', 'ולפי', 'מה', 'עדיף', 'ללכת', ',', 'זה', 'של', 'שנה', 'הבאה', 'באוסטרליה', 'או', 'זה', 'של', 'השנה', 'בבלגיה', '16.12.2024', ',', '16:45', '-', '<', 'המדיה', 'לא', 'נכללה', '>', '16.12.2024', ',', '18:03', '-', 'אוסטרליה', '25', '.', 'את', 'התקנון', 'שאנחנו', 'נתחרה', 'איתו', 